In [1]:
import requests 
from bs4 import BeautifulSoup
from pymongo import MongoClient
from datetime import datetime
from tqdm import tqdm
import random 
import time

In [2]:
mongo = MongoClient("mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&ssl=false")
mongo_col = mongo["delfi_texts"]["lv_news"]

In [3]:
def get_date(soup):
    date_tag = soup.find("time")
    date = None
    if date_tag != None:
        date_s = date_tag.text
        date_s = date_s.replace("\t", "")
        date_s = date_s.replace("\n", "")
        if date_s.startswith("  "):
            date_s = date_s.replace("  ", "")
        try:
            date = datetime.strptime(date_s, "%d.%m.%Y %H:%M")
        except Exception as e:
            try:
                date = datetime.strptime(date_s, "%Y.%m.%d %H:%M:%S")
            except Exception as e2:
                print(current_link)
                print(e)
                print(e2)
    return date

def get_text(soup):
    text = ""
    #lead = soup.find("div", {"class": "delfi-article-lead"}).text
    #text += lead
    p_tags = soup.find_all("p")
    for p in p_tags:
        if not p.text.endswith("..."):
            text += p.text+"\n "
    return text
    
def clean_link(link):
    bad_parts = ["https://www.facebook.com/sharer/sharer.php?u=", 
                 "https://www.facebook.com/dialog/send?app_id=1476286675946138&link=", 
                 "https://www.facebook.com/dialog/send?app_id=1476286675946138&link=",
                 "&com=1"]
    for bp in bad_parts:
        link = link.replace(bp, "")
    #link = link.split("&redirect_uri=")[0]
    link = link.split("&")[0]
    return link

def get_links(soup):
    a_tags = soup.find_all("a")
    links = [i for i in a_tags if "href" in i.attrs]
    rel_links = set([i["href"] for i in links if i["href"].startswith(core_link)])
    rel_links2 = set([clean_link(i) for i in rel_links])
    return rel_links2

def add_to_visited(link):
    last_char = link[-1]
    if last_char == "0":
        visited_0.add(link)
    elif last_char == "1":
        visited_1.add(link)
    elif last_char == "2":
        visited_2.add(link)
    elif last_char == "3":
        visited_3.add(link)
    elif last_char == "4":
        visited_4.add(link)
    elif last_char == "5":
        visited_5.add(link)
    elif last_char == "6":
        visited_6.add(link)
    elif last_char == "7":
        visited_7.add(link)
    elif last_char == "8":
        visited_8.add(link)
    elif last_char == "9":
        visited_9.add(link)
    else :
        visited_oth.add(link)
        
        
def check_visited(link):
    last_char = link[-1]
    
    if last_char == "0":
        return link in visited_0
    elif last_char == "1":
        return link in visited_1
    elif last_char == "2":
        return link in visited_2
    elif last_char == "3":
        return link in visited_3
    elif last_char == "4":
        return link in visited_4
    elif last_char == "5":
        return link in visited_5
    elif last_char == "6":
        return link in visited_6
    elif last_char == "7":
        return link in visited_7
    elif last_char == "8":
        return link in visited_8
    elif last_char == "9":
        return link in visited_9
    else:
        return link in visited_oth
    

In [4]:
core_link = "https://www.delfi.lv/news/national/politics/"


In [5]:
visited_0 = set()
visited_1 = set()
visited_2 = set()
visited_3 = set()
visited_4 = set()
visited_5 = set()
visited_6 = set()
visited_7 = set()
visited_8 = set()
visited_9 = set()
visited_oth = set()

In [6]:
query = {}
projection = {"_id" : 0, "source":1}

for i in tqdm(mongo_col.find(query, projection)):
    add_to_visited(i["source"])

138006it [00:01, 115164.41it/s]


In [7]:
#initial
links = set()

res = requests.get(core_link)
add_to_visited(core_link)
if res.status_code == 200 :
    soup = BeautifulSoup(res.text, "html5lib")
    site_links = get_links(soup)
    for link in site_links:
        if check_visited(link) == False:
            links.add(link)
else:
    pass

In [8]:
list(links)[0:4]

['https://www.delfi.lv/news/national/politics/zanda-kalnina-lukasevica-es-ir-japieskir-kandidatvalsts-statuss-ukrainai.d?id=54122056',
 'https://www.delfi.lv/news/national/politics/video-gajiens-kopa-ar-ukrainu-kopa-pret-putinu-pulcejis-30-000-cilveku.d?id=54122464',
 'https://www.delfi.lv/news/national/politics/rigas-dome-lidz-sim-pieteikti-divi-16-marta-pasakumi.d?id=54120178',
 'https://www.delfi.lv/news/national/politics/sandi-girgenu-ievel-par-politiskas-partijas-republika-priekssedetaju.d?id=54122716']

In [9]:
#last_page = 6210

#pages = [last_page - i for i in range(0, last_page)]

#random.shuffle(pages)

pages = [i for i in range(2, 350)]

pages = pages[0:1000]

pages[0:5]

[2, 3, 4, 5, 6]

In [10]:
link_fragment = "https://www.delfi.lv/news/national/politics/?page="
#"""
for i in tqdm(pages):
    res = requests.get(link_fragment+str(i))
    if res.status_code == 200 :
        soup = BeautifulSoup(res.text, "html5lib")
        site_links = get_links(soup)
        for link in site_links:
            if check_visited(link) == False:
                links.add(link)
    else:
        pass
    sleep_interval = random.randint(100,300)/100
    time.sleep(sleep_interval)
#"""

100%|██████████| 348/348 [19:10<00:00,  3.31s/it]


In [11]:
print(len(links))

582


In [12]:
counter = 0
faulty_links = []
print(datetime.now(), " Started")
while True :
    if len(links) > 0:
        current_link = links.pop()
        add_to_visited(current_link)
        try:
            res = requests.get(current_link)
            if res.status_code == 200 :
                try:
                    soup = BeautifulSoup(res.text, "html5lib")
                    date = get_date(soup)
                    site_links = get_links(soup)
                    text = get_text(soup)

                    doc = {"source" : current_link, 
                           "text": text, 
                           "date": date }
                    mongo_col.insert_one(doc)
                    counter +=1 

                    if counter % 1000 == 0:
                        print(datetime.now(), " Counter at ", counter)

                    for link in site_links:
                        if check_visited(link) == False:
                            links.add(link)

                except Exception as e:
                    print(datetime.now(), current_link, e)   
                    faulty_links.append(current_link)
        except Exception as e:
            print(datetime.now(), " Faulty link ", current_link)
        
        pass
    else:
        if len(faulty_links) > 0:
            for link in faulty_links:
                links.add(link)
        else:
            break
            

2022-03-05 16:29:35.933633  Started


In [ ]:
date

In [ ]:
current_link = "https://www.delfi.lv/news/national/politics/deputate-ir-istais-laiks-debatem-par-deputatu-un-prezidenta-impicmenta-iespejam.d?id=42485970"

res = requests.get(current_link)
if res.status_code == 200 :
    try:
        soup = BeautifulSoup(res.text, "html5lib")
        date = get_date(soup)
        site_links = get_links(soup)
        text = get_text(soup)

        doc = {"source" : current_link, 
               "text": text, 
               "date": date }
    except Exception as e:
        print(e)

In [ ]:
date_tag = soup.find("time")
date = None
if date_tag != None:
    date_s = date_tag.text
    date_s = date_s.replace("\t", "")
    date_s = date_s.replace("\n", "")
    if date_s.startswith("  "):
        date_s = date_s.replace("  ", "")
    try:
        date = datetime.strptime(date_s, "%d.%m.%Y %H:%M")
    except Exception as e:
        try:
            date = datetime.strptime(date_s, "%Y.%m.%d %H:%M:%S")
        except Exception as e2:
            print(current_link)
            print(e)
            print(e2)

In [ ]:
date_s.replace("  ", "")

In [ ]:
text = ""
#lead = soup.find("div", {"class": "delfi-article-lead"}).text
#text += lead
p_tags = soup.find_all("p")
for p in p_tags:
    if not p.text.endswith("..."):
        text += p.text+"\n "

In [ ]:
text

In [ ]:
def get_date(soup:Object)->datetime:
    """Extracts publish time from the page source"""
    date_tag = soup.find("meta", {"name": "cXenseParse:recs:publishtime"})
    date = None
    if date_tag != None:
        if "content" in date_tag.attrs:
            date_s = date_tag["content"]
            date = datetime.strptime(date_s.split("+")[0], "%Y-%m-%dT%H:%M:%S")
    return date

In [ ]:
def get_date(soup):
    date_tag = soup.find("time")
    date = None
    if date_tag != None:
        date_s = date_tag.text
        date_s = date_s.replace("\t", "")
        date_s = date_s.replace("\n", "")
        date = datetime.strptime(date_s, "%d.%m.%Y %H:%M")
    return date

In [ ]:
get_date(soup)

In [ ]:
date_tag = soup.find("time")
date = None
if date_tag != None:
    date_s = date_tag.text
    date_s = date_s.replace("\t", "")
    date_s = date_s.replace("\n", "")
    date = datetime.strptime(date_s, "%d.%m.%Y %H:%M")
date

In [ ]:
soup

In [ ]:
"%d.%m.%Y %H:%M"